In [1]:
!pip install geopandas
!pip install geopy

    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\Asus\anaconda3\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Asus\\AppData\\Local\\Temp\\pip-install-qokfspef\\fiona\\setup.py'"'"'; __file__='"'"'C:\\Users\\Asus\\AppData\\Local\\Temp\\pip-install-qokfspef\\fiona\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\Asus\AppData\Local\Temp\pip-pip-egg-info-b7so1ufv'
         cwd: C:\Users\Asus\AppData\Local\Temp\pip-install-qokfspef\fiona\
    Complete output (1 lines):
    A GDAL API version must be specified. Provide a path to gdal-config using a GDAL_CONFIG environment variable or use a GDAL_VERSION environment variable.
    ----------------------------------------
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


  Using cached geopandas-0.9.0-py2.py3-none-any.whl (994 kB)


In [2]:
import pandas as pd
from geopy.geocoders import Nominatim

In [3]:
df = pd.read_excel('clean leads.xlsx',dtype='str')


FileNotFoundError: [Errno 2] No such file or directory: 'clean leads.xlsx'

In [11]:
locator = Nominatim(user_agent="myGeocoder")

In [21]:
df_copy = df.copy()
df_copy=df_copy.dropna(subset=['Add']).heads
df_copy

Source              Name       Phone  \
3448  11. 3400 DSKH mua Times City.xlsx   Trần Ngọc Thanh  0913082526   
3449  11. 3400 DSKH mua Times City.xlsx  Nguyễn Khánh Chi  0913958338   
3450  11. 3400 DSKH mua Times City.xlsx  Nguyễn Minh Thùy  0916058730   
3451  11. 3400 DSKH mua Times City.xlsx      Phạm Quốc Sử  0989313246   
3452  11. 3400 DSKH mua Times City.xlsx  Nguyễn Mạnh Hùng  0989892276   
3453  11. 3400 DSKH mua Times City.xlsx   Nguyễn Văn Long  0945393999   

          Phone2 Phone3 Birthdate                       Email  \
3448         NaN    NaN       NaN          thanhtn@acb.com.vn   
3449  0944473333    NaN       NaN      nguyenchi154@gmail.com   
3450         NaN    NaN       NaN  nguyenminhthuy82@yahoo.com   
3451         NaN    NaN       NaN      phamquocsu82@gmail.com   
3452         NaN    NaN       NaN           hungbtc@yahoo.com   
3453         NaN    NaN       NaN       longlbtt@yahoo.com.vn   

                                                    Add    City  
3448                    75 Lê Duẩn, Q.Hoàn Kiếm, Hà Nội  Hà Nội  
3449  154 đường Nghi Tàm, phường Yên Phụ, Quận Tây H...  Hà Nội  
3450  Phòng 23 tầng 2 tập thể 31 Thi Sách , Hai Bà T...  Hà Nội  
3451  Số nhà 38 đường 11 tập thể F361 An Dương, Yên ...  Hà Nội  
3452  505-E2 Hồ Việt Xô, Bạch Đằng, Hai Bà Trưng, Hà...  Hà Nội  
3453   Thôn Hoài Thượng, xã Liên Bão, Tiên Du, Bắc Ninh     NaN

In [22]:
from geopy.extra.rate_limiter import RateLimiter

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df_copy['location'] = df_copy['Add'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df_copy['point'] = df_copy['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df_copy[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df_copy['point'].tolist(), index=df_copy.index)

In [33]:
df_copy

Source              Name       Phone  \
3448  11. 3400 DSKH mua Times City.xlsx   Trần Ngọc Thanh  0913082526   
3449  11. 3400 DSKH mua Times City.xlsx  Nguyễn Khánh Chi  0913958338   

          Phone2 Phone3 Birthdate                   Email  \
3448         NaN    NaN       NaN      thanhtn@acb.com.vn   
3449  0944473333    NaN       NaN  nguyenchi154@gmail.com   

                                                    Add    City  \
3448                    75 Lê Duẩn, Q.Hoàn Kiếm, Hà Nội  Hà Nội   
3449  154 đường Nghi Tàm, phường Yên Phụ, Quận Tây H...  Hà Nội   

                                               location  \
3448  (Đường Lê Duẩn, Cửa Nam, Phường Cửa Nam, Quận ...   
3449  (Đường Nghi Tàm, Yên Phụ, Phường Yên Phụ, Quận...   

                               point   latitude   longitude  altitude  
3448     (21.0261422, 105.8415, 0.0)  21.026142  105.841500       0.0  
3449  (21.0573869, 105.8354464, 0.0)  21.057387  105.835446       0.0

In [26]:
!pip install folium
import folium

In [29]:
map1 = folium.Map(
    location=[21.0245, 105.84117],
    tiles='cartodbpositron',
    zoom_start=12,
)
df_copy.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1